# Exploratory data analysis for Scicore- DataEngineering 

Inital data exploration of the 4 datasets and rlevant attributes

In [159]:
import pandas as pd
import pydantic
from typing import List, Tuple
from datetime import datetime
from pathlib import Path
from io import StringIO
from Bio import SeqIO

## Illumina data

relevant columns are 1) sample number, 2) path to R1 file, 3) path to R2 file

## description

By looking at the illumina.tree file it is clear there are several different types of files:    
1. R1/R2 "fq.qz" files which should be FASTQ genetic sequencing data
2. R1/R2 "trimming_report.txt"  files which could be reports on the trimming operation to create the "short reads" genetic sequencing
3. slurm log file from the trimSlurmReports, probably related to some preprocessing operation on the data. It seems there is no easy way to connect the slurmreports to the sequenecing files above

I will create a dataframe for each file with the following columns:
sample_id, sequence, length, read_type, filepath only for the FASTQ data, exluding trimming reports and slurm jobs


In [74]:
# pydantic data model for illumina data

class IlluminaDataModel(pydantic.BaseModel):
    sample_id: str
    sample_id_suffix: str
    seq: str
    sample_len: str
    read_type: str
    filename: str

    @pydantic.validator('sample_id')
    def clean_sample_id(cls,v):
        """ cleans the sample id removing non standard characters"""
        clean_id = v.split(" ")[-1]
        if not all([a.isnumeric() for a in clean_id.split("-")]):
            raise ValueError(f"sample id is non standard: {clean_id}")
        else:
            return clean_id.split("-")[0]
        
    @pydantic.validator('sample_id_suffix')
    def clean_sample_id_suffix(cls,v):
        """ cleans the sample id removing non standard characters"""
        clean_id = v.split(" ")[-1]
        if not all([a.isnumeric() for a in clean_id.split("-")]):
            raise ValueError(f"sample id is non standard: {clean_id}")
        else:
            return "-".join(clean_id.split("-")[1:])
        

    @pydantic.validator('seq')
    def check_sequence(cls,v):
        """ checks that the sequence is in format letter:digit(s)"""
        if (v[0].isnumeric()) or (not v[1:].isnumeric()):
            raise ValueError( f"sequence is non standard: {v}")
        else:
            return v
        
    @pydantic.validator('sample_len')
    def check_length(cls,v):
        """ checks that the sequence is in format letter:digit(s)"""
        if (v[0].isnumeric()) or (not v[1:-1].isnumeric()):
            raise ValueError( f"sequence is non standard: {v}")
        else:
            return v
        
    @pydantic.validator('read_type')
    def clean_read_type(cls,v):
        """ checks that the sequence is in format letter:digit(s)"""
        if v[0].isnumeric() or (not v[1:].isnumeric()):
            raise ValueError( f"sequence is non standard: {v}")
        else:
            return int(v[1:])

    
    @pydantic.validator('filename')
    def check_filename(cls, v):
        """check filename for correct extension ans beginning"""
        clean_fname= v.split(" ")[-1]
        if (not clean_fname.endswith(".gz")) or (not clean_fname[0].isnumeric()):
            raise ValueError(f"filename is non standard: {clean_fname}")
        else:
            return clean_fname
        

In [75]:
def import_and_check_ill_data(
    fp: str, dataModel: pydantic.BaseModel
) -> Tuple[List[pydantic.BaseModel], List[dict]]:
    """opens a tree file and imports the data following given
    pydantic data model

    Returns:
    list of objects as per data model, list of errors

    """
    data = []
    errors = []
    # Open the .tree file and read its contents
    with open(fp, "r") as f:
        for line in f:
            line = line.rstrip("\n")

            if line.endswith("gz"):
                components = line.strip().split("_")
                try:
                    obj = dataModel(
                        sample_id=components[0],
                        sample_id_suffix=components[0],
                        seq=components[1],
                        sample_len=components[2],
                        read_type=components[3],
                        filename=line,
                    )
                    data.append(obj)
                except pydantic.ValidationError as e:
                    errors.append({"line": line, "error": str(e)})

    return data, errors


# ----
fp = "../illumina_data.tree"
objs, errors = import_and_check_ill_data(fp, IlluminaDataModel)
print(f"total number of objects imported correctly: {len(objs)}")
print(f"total number of errors: {len(errors)}")


total number of objects imported correctly: 336
total number of errors: 0


In [76]:
# load illumina_data as dataframe for exploration
ill_df = pd.DataFrame([a.dict() for a in objs])
ill_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 336 entries, 0 to 335
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   sample_id         336 non-null    object
 1   sample_id_suffix  336 non-null    object
 2   seq               336 non-null    object
 3   sample_len        336 non-null    object
 4   read_type         336 non-null    int64 
 5   filename          336 non-null    object
dtypes: int64(1), object(5)
memory usage: 15.9+ KB


## Nanopore data

relevant columns are 1) sequencing date, 2) path to sequencing files (one per row)

In [79]:
# nanopore datamodel
class NanoporeDataModel(pydantic.BaseModel):
    seq_date: str
    barcode_number: str
    path_to_seq_file: str

    @pydantic.validator("seq_date")
    def transform_date(cls, v):
        """transforms str to date"""
        clean_date = datetime.strptime(v, "%Y%m%d")
        return clean_date

    @pydantic.validator("barcode_number")
    def transform_barcode(cls, v):
        if not v.isnumeric():
            raise ValueError(f"barcode is not numeric: {v}")
        return int(v)

    @pydantic.validator("path_to_seq_file")
    def check_path(cls, v):
        """checks that path is actually a consistent path using pathlib"""
        my_path = Path(v)
        return v


In [80]:
def load_and_check_nanopore_data(
    filepath: str, dataModel: pydantic.BaseModel
) -> Tuple[List[pydantic.BaseModel], List[dict]]:
    """loads nanopore data from source file checking consistency using the given pydantic data model"""

    data = []
    errors = []
    # Open the .tree file and read its contents
    with open(filepath, "r") as f:
        for line in f:
            line = line.rstrip("\n")
            if line.startswith("nanopore"):
                parent = line
                seq_date = parent.split("/")[1].split("_")[0]
            else:
                components = line.split(" ")[-1].split(".")
                try:
                    obj = dataModel(
                        seq_date=seq_date,
                        barcode_number=components[0][-2:],
                        path_to_seq_file=f"{parent}/{line.split(' ')[-1]}",
                    )
                    data.append(obj)
                except pydantic.ValidationError as e:
                    errors.append({"line": line, "date": seq_date, "error": str(e)})
    return data, errors


# ---

filepath = "../nanopore_data.tree"
objs, errors = load_and_check_nanopore_data(filepath, NanoporeDataModel)
print(f"total number of objects imported correctly: {len(objs)}")
print(f"total number of errors: {len(errors)}")


total number of objects imported correctly: 194
total number of errors: 20


In [81]:
nano_df = pd.DataFrame([a.dict() for a in objs])
nano_df.info()
nano_df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194 entries, 0 to 193
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   seq_date          194 non-null    datetime64[ns]
 1   barcode_number    194 non-null    int64         
 2   path_to_seq_file  194 non-null    object        
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 4.7+ KB


,seq_date,barcode_number,path_to_seq_file
0,2018-01-04,5,nanopore/20180104_Carbapenemases/porechop/BC05...
1,2018-01-04,6,nanopore/20180104_Carbapenemases/porechop/BC06...
2,2018-01-04,7,nanopore/20180104_Carbapenemases/porechop/BC07...
3,2018-01-04,8,nanopore/20180104_Carbapenemases/porechop/BC08...
4,2018-01-04,9,nanopore/20180104_Carbapenemases/porechop/BC09...


In [82]:
for e in errors:
    print(e["date"][-20:], e["error"], "\n", e["line"][-20:])


20180104 1 validation error for NanoporeDataModel
barcode_number
  barcode is not numeric: ne (type=value_error) 
 └── none.fastq.gz
20180108 1 validation error for NanoporeDataModel
barcode_number
  barcode is not numeric: ne (type=value_error) 
 └── none.fastq.gz
20180110 1 validation error for NanoporeDataModel
barcode_number
  barcode is not numeric: ne (type=value_error) 
 └── none.fastq.gz
20180118 1 validation error for NanoporeDataModel
barcode_number
  barcode is not numeric: ne (type=value_error) 
 └── none.fastq.gz
20180123 1 validation error for NanoporeDataModel
barcode_number
  barcode is not numeric: ne (type=value_error) 
 └── none.fastq.gz
20180126 1 validation error for NanoporeDataModel
barcode_number
  barcode is not numeric: ne (type=value_error) 
 ├── none.fastq.gz
20180126 1 validation error for NanoporeDataModel
barcode_number
  barcode is not numeric: es (type=value_error) 
 └── samples.txt
20180208 1 validation error for NanoporeDataModel
barcode_number
  barc

## MIC data

relevant columns are 1) sample number, 2) path to R1 file, 3) path to R2 file

In [83]:
fp = "../MIC_data.csv"

mic_df = pd.read_csv(fp)

mic_df.head()


,strain,Isolate #,Species,Labor Number,Date,Box,Platz,DNA concentration,Date of isolation,Localisation,...,Norfloxacin,Ciprofloxacin,Levofloxacin,Colistin,Fosfomycin-Trometamol,Tigecyclin,Rifampicin,Minocyclin,Doxycyclin,comment
0,carb001,38,Kleb,402738,2010,1.0,A2,27.4,11/15/2010,unknown (culture),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Phenotypic evidence for carbapenemase; KPC, VI..."
1,carb002,48,Kleb,402859,2010,1.0,A6,18.2,12/1/2010,unknown (culture),...,>=16,>=4,NaN,NaN,>=256,NaN,NaN,NaN,NaN,"Phenotypic evidence for carbapenemase; KPC, VI..."
2,carb003,14,Esch,800329,2011,1.0,B3,57.6,1/26/2011,"Swab, inguinal",...,>=16,>=4,NaN,0.125,<=16,NaN,NaN,NaN,NaN,no genotypic PCR result available
3,carb004,41,Kleb,400793,2011,1.0,B5,39.6,3/22/2011,unknown (culture),...,NaN,NaN,NaN,NaN,<=16,NaN,NaN,NaN,NaN,KPC in PCR detected
4,carb005,31,Kleb,704984,2011,1.0,B6,28.6,3/28/2011,Urine,...,>=16,>=4,NaN,0.5,>=256,NaN,NaN,NaN,NaN,Phenotypic ESBL and porinlost


## Sample TABLE

relevant columns are 1) sample number, 2) path to R1 file, 3) path to R2 file

In [84]:
fp = "../sample_table.csv"

sample_df = pd.read_csv(fp)

sample_df.head()


,Internal #,Isolate #,Species,Labor Number,Date,Box,Platz,DNA Konzentration,sequenced,mb,barcode,resequenced,rebarcode
0,1,38,Kleb,402738,2010,1.0,A2,27.4,20180104,NaN,5,NaN,NaN
1,2,48,Kleb,402859,2010,1.0,A6,18.2,20180123,NaN,7,20180420.0,2.0
2,3,14,Esch,800329,2011,1.0,B3,57.6,20180104,NaN,6,NaN,NaN
3,4,41,Kleb,400793,2011,1.0,B5,39.6,20180104,NaN,7,NaN,NaN
4,5,31,Kleb,704984,2011,1.0,B6,28.6,20180104,NaN,8,NaN,NaN


# exploring uniProt api

Here we build a query for the restapi of uniprot, testing to get data


In [98]:
import requests

url = "https://rest.uniprot.org/uniprotkb/stream?compressed=false&format=fasta&query=%28%28taxonomy_id%3A573%29%20AND%20Imipenem%29"
all_fastas = requests.get(url).text


In [161]:
def fasta_to_df(fasta_str:str)->pd.DataFrame:
    
    fasta_io = StringIO(fasta_str) 
    records = SeqIO.parse(fasta_io, "fasta")

    output_list= []
    for r in records:
        fasta_dict ={
        'id': r.id,
        'description': r.description,
        'name': r.name,
        }
        output_list.append(fasta_dict)

    output_df =pd.DataFrame(output_list)


#--
fasta_to_df(all_fastas)


,id,description,name,seq
0,tr|A0A0A8IKC4|A0A0A8IKC4_KLEPN,tr|A0A0A8IKC4|A0A0A8IKC4_KLEPN Bifunctional NA...,tr|A0A0A8IKC4|A0A0A8IKC4_KLEPN,"(M, M, D, H, T, M, T, K, N, P, D, S, I, P, Y, ..."
1,sp|C7C422|BLAN1_KLEPN,sp|C7C422|BLAN1_KLEPN Metallo-beta-lactamase t...,sp|C7C422|BLAN1_KLEPN,"(M, E, L, P, N, I, M, H, P, V, A, K, L, S, T, ..."
2,tr|Q6XEC0|Q6XEC0_KLEPN,tr|Q6XEC0|Q6XEC0_KLEPN Beta-lactamase OS=Klebs...,tr|Q6XEC0|Q6XEC0_KLEPN,"(M, R, V, L, A, L, S, A, V, F, L, V, A, S, I, ..."
3,tr|Q79PC4|Q79PC4_KLEPN,tr|Q79PC4|Q79PC4_KLEPN Dihydropteroate synthas...,tr|Q79PC4|Q79PC4_KLEPN,"(M, V, T, V, F, G, I, L, N, L, T, E, D, S, F, ..."
4,tr|Q7BK05|Q7BK05_KLEPN,tr|Q7BK05|Q7BK05_KLEPN EmrE OS=Klebsiella pneu...,tr|Q7BK05|Q7BK05_KLEPN,"(M, K, G, W, L, F, L, V, I, A, I, V, G, E, V, ..."
...,...,...,...,...
74,tr|Q6SLK9|Q6SLK9_KLEPN,tr|Q6SLK9|Q6SLK9_KLEPN IS1380-like element ISE...,tr|Q6SLK9|Q6SLK9_KLEPN,"(M, I, N, K, I, D, F, K, A, K, N, L, T, S, N, ..."
75,tr|Q6XEB9|Q6XEB9_KLEPN,tr|Q6XEB9|Q6XEB9_KLEPN LysR OS=Klebsiella pneu...,tr|Q6XEB9|Q6XEB9_KLEPN,"(M, P, D, L, N, G, M, M, L, F, A, A, V, V, R, ..."
76,tr|Q6XEC1|Q6XEC1_KLEPN,tr|Q6XEC1|Q6XEC1_KLEPN TnpA OS=Klebsiella pneu...,tr|Q6XEC1|Q6XEC1_KLEPN,"(M, R, E, A, N, I, L, Q, H, S, L, H, Q, Y, C, ..."
77,tr|Q767E2|Q767E2_KLEPN,tr|Q767E2|Q767E2_KLEPN IS6100 Transposase OS=K...,tr|Q767E2|Q767E2_KLEPN,"(M, L, G, G, D, W, T, D, G, T, M, T, D, F, K, ..."
